# Predicting Demonstration Size on Demographic Characteristics and Salience

## GW DATS 6103: Introduction to Data Mining Final Project
### Name: Alexander D. Silberman

Note: Data is limited to 2017–2019, as COVID may substantially throw off the data.

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

from census import Census
from us import states

import censusdis.data as ced
from censusdis.datasets import ACS1_PROFILE
from censusdis import states

## Data Acquisition and Wrangling

### Results Dataset: Crowd Counting Consortium Phase 1 (2017-2020)

Our results dataset: Crowd Counting Consortium Phase 1 (2017-2020): https://ash.harvard.edu/programs/crowd-counting-consortium/

In [130]:
crowd_data = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")
crowd_data.head(5)

C:\Users\alexa\AppData\Local\Temp\ipykernel_41792\206043875.py:1: DtypeWarning: Columns (6,22,31,32,33,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  crowd_data = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")


,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,source_28,source_29,source_30,notes,lat,lon,resolved_locality,resolved_county,resolved_state,fips_code
0,2017-01-01,Washington,DC,Lafayette Square Park,0.0,vigil,NaN,NaN,"for banning nuclear weapons, for peace",0.0,...,NaN,NaN,NaN,White House Peace Vigil continuous since June ...,38.907192,-77.036871,Washington,District of Columbia,DC,11001.0
1,2017-01-01,Mankato,MN,NaN,0.0,vigil,NaN,Peace Vigil Mankato,for peace,0.0,...,NaN,NaN,NaN,every Sunday since 2001,44.163578,-93.999400,Mankato,Blue Earth County,MN,27013.0
2,2017-01-01,Minneapolis,MN,U.S. Bank Stadium,0.0,protest; banner drop,NaN,general protestors,"against the Dakota Access Pipeline, for indige...",1.0,...,NaN,NaN,NaN,hung banner from stadium roof during NFL game,44.977753,-93.265011,Minneapolis,Hennepin County,MN,27053.0
3,2017-01-01,Little Compton,RI,Town Green,0.0,vigil,NaN,Sakonnet Peace Alliance,"for peace, for gun control, for climate action",1.0,...,NaN,NaN,NaN,every Sunday since 2003,41.510103,-71.171156,Little Compton,Newport County,RI,44005.0
4,2017-01-01,Oak Ridge,TN,Y-12 National Security Complex,0.0,vigil,NaN,Oak Ridge Environmental Peace Alliance,for abolishing nuclear weapons,0.0,...,NaN,NaN,NaN,every Sunday since the late 1990s,36.010356,-84.269645,Oak Ridge,Anderson County,TN,47001.0


In [144]:
source_cols = np.array(["source_"]*30) + np.arange(1,31).astype(str)
crowd_data.drop(columns=source_cols, inplace=True)
crowd_data.head(5)
# crowd_data["fips_code"] = crowd_data["fips_code"].astype(np.int16)
# crowd_data["fips_code"]


,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,property_damage,property_damage_any,chemical_agents,notes,lat,lon,resolved_locality,resolved_county,resolved_state,fips_code
0,2017-01-01,Washington,DC,Lafayette Square Park,0.0,vigil,NaN,NaN,"for banning nuclear weapons, for peace",0.0,...,NaN,0,0.0,White House Peace Vigil continuous since June ...,38.907192,-77.036871,Washington,District of Columbia,DC,11001.0
1,2017-01-01,Mankato,MN,NaN,0.0,vigil,NaN,Peace Vigil Mankato,for peace,0.0,...,NaN,0,0.0,every Sunday since 2001,44.163578,-93.999400,Mankato,Blue Earth County,MN,27013.0
2,2017-01-01,Minneapolis,MN,U.S. Bank Stadium,0.0,protest; banner drop,NaN,general protestors,"against the Dakota Access Pipeline, for indige...",1.0,...,NaN,0,NaN,hung banner from stadium roof during NFL game,44.977753,-93.265011,Minneapolis,Hennepin County,MN,27053.0
3,2017-01-01,Little Compton,RI,Town Green,0.0,vigil,NaN,Sakonnet Peace Alliance,"for peace, for gun control, for climate action",1.0,...,NaN,0,0.0,every Sunday since 2003,41.510103,-71.171156,Little Compton,Newport County,RI,44005.0
4,2017-01-01,Oak Ridge,TN,Y-12 National Security Complex,0.0,vigil,NaN,Oak Ridge Environmental Peace Alliance,for abolishing nuclear weapons,0.0,...,NaN,0,0.0,every Sunday since the late 1990s,36.010356,-84.269645,Oak Ridge,Anderson County,TN,47001.0


In [149]:
print(len(crowd_data[crowd_data["fips_code"].isna()]), "out of", len(crowd_data))

107 out of 72181


Only 107 unresolved localities out of 72181; we may safely discard these.

In [155]:
crowd_data = crowd_data[crowd_data["fips_code"].notna()]
crowd_data["fips_code"] = crowd_data["fips_code"].astype(int)
crowd_data["fips_code"]

0        11001
1        27013
2        27053
3        44005
4        47001
         ...  
72176    48029
72177    48029
72178    51710
72179    53033
72180    53033
Name: fips_code, Length: 72074, dtype: int64

### American Community Survey

In [67]:
api_key = "29e7dfea2f8b253a0a10ccd9626f78e49f4f0a4f"

def regex_filter(string, myregex):
    if string:
        mo = re.search(myregex, string)
        if mo:
            return True
        else:
            return False
    else:
        return False

census_vars = ced.variables.search(ACS1_PROFILE, 2017)
census_vars = census_vars[census_vars["GROUP"] != "N/A"]
census_vars_pcts = census_vars[census_vars["VARIABLE"].apply(regex_filter,myregex=r"PE$")]
census_vars_pcts

,YEAR,DATASET,GROUP,VARIABLE,LABEL,SUGGESTED_WEIGHT,VALUES
11,2017,acs/acs1/profile,DP02PR,DP02PR_0001PE,Percent!!HOUSEHOLDS BY TYPE!!Total households,NaN,None
13,2017,acs/acs1/profile,DP02PR,DP02PR_0002PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
15,2017,acs/acs1/profile,DP02PR,DP02PR_0003PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
17,2017,acs/acs1/profile,DP02PR,DP02PR_0004PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
19,2017,acs/acs1/profile,DP02PR,DP02PR_0005PE,Percent!!HOUSEHOLDS BY TYPE!!Total households!...,NaN,None
...,...,...,...,...,...,...,...
1347,2017,acs/acs1/profile,DP05,DP05_0085PE,Percent!!HISPANIC OR LATINO AND RACE!!Total po...,NaN,None
1349,2017,acs/acs1/profile,DP05,DP05_0086PE,Percent!!Total housing units,NaN,None
1351,2017,acs/acs1/profile,DP05,DP05_0087PE,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...",NaN,None
1353,2017,acs/acs1/profile,DP05,DP05_0088PE,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...",NaN,None


Solely request percents. The list of variables includes what should be the total population as a percent—`DP05_0001P`—but, as that would always be 100%, the wise people at the US Census Bureau made such equal to `DP05_0001E`, the total population estimate, necessitating no further adjustment.

In [58]:
ced.variables.group_tree(ACS1_PROFILE, 2017,"DP02PR")

+ Estimate
    + HOUSEHOLDS BY TYPE
        + Total households (DP02PR_0001E)
            + Family households (families) (DP02PR_0002E)
                + With own children of the householder under 18 years (DP02PR_0003E)
                + Married-couple family (DP02PR_0004E)
                    + With own children of the householder under 18 years (DP02PR_0005E)
                + Male householder, no wife present, family (DP02PR_0006E)
                    + With own children of the householder under 18 years (DP02PR_0007E)
                + Female householder, no husband present, family (DP02PR_0008E)
                    + With own children of the householder under 18 years (DP02PR_0009E)
            + Nonfamily households (DP02PR_0010E)
                + Householder living alone (DP02PR_0011E)
                    + 65 years and over (DP02PR_0012E)
        + Households with one or more people under 18 years (DP02PR_0013E)
        + Households with one or more people 65 years and over (

In [111]:
census_data = [ced.download(ACS1_PROFILE, year, pd.concat([pd.Series(["NAME"]), census_vars_pcts["VARIABLE"]], ignore_index=True), 
                           metropolitan_statistical_area_micropolitan_statistical_area ="*", 
                           api_key=api_key
                           ).set_index("METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STATISTICAL_AREA")
               for year in range(2017,2020)]


census_data = pd.concat(census_data, keys = np.arange(2017,2020))
census_data

NAME  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                                                 
2017 44220                                                                 Springfield, OH Metro Area   
     44300                                                               State College, PA Metro Area   
     44340                                                                  Statesboro, GA Micro Area   
     44420                                                         Staunton-Waynesboro, VA Metro Area   
     44620                                                               Stevens Point, WI Micro Area   
...                                                                                               ...   
2019 49620                                                                York-Hanover, PA Metro Area   
     49660                                               Youngstown-Warren-Boardman, OH-PA Metro Area   
     49700                                                                   Yuba City, CA Metro Area   
     49740                                                                        Yuma, AZ Metro Area   
     49780                                                                  Zanesville, OH Micro Area   

                                                         DP02PR_0001PE  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                  
2017 44220                                                         NaN   
     44300                                                         NaN   
     44340                                                         NaN   
     44420                                                         NaN   
     44620                                                         NaN   
...                                                                ...   
2019 49620                                                         NaN   
     49660                                                         NaN   
     49700                                                         NaN   
     49740                                                         NaN   
     49780                                                         NaN   

                                                         DP02PR_0002PE  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                  
2017 44220                                                         NaN   
     44300                                                         NaN   
     44340                                                         NaN   
     44420                                                         NaN   
     44620                                                         NaN   
...                                                                ...   
2019 49620                                                         NaN   
     49660                                                         NaN   
     49700                                                         NaN   
     49740                                                         NaN   
     49780                                                         NaN   

                                                         DP02PR_0003PE  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                  
2017 44220                                                         NaN   
     44300                                                         NaN   
     44340                                                         NaN   
     44420                                                         NaN   
     44620                                                         NaN   
...                                                                ...   
2019 49620                                                         NaN   
     49660                                                         NaN   
     49700                                                       

In [109]:
metro_micro_area_text = re.compile(r"(\s(Metro|Micro)\sArea)$")
census_data["NAME"].apply(lambda x: metro_micro_area_text.sub("",x))
# re.sub(string="New York Metro Area", pattern=r"(Metro|Micro)\sArea$", repl="hello")

      METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STATISTICAL_AREA
2017  44220                                                                            Springfield, OH
      44300                                                                          State College, PA
      44340                                                                             Statesboro, GA
      44420                                                                    Staunton-Waynesboro, VA
      44620                                                                          Stevens Point, WI
                                                                                   ...                
2019  49620                                                                           York-Hanover, PA
      49660                                                          Youngstown-Warren-Boardman, OH-PA
      49700                                                                              Yuba City, CA
      4

### Most Important Problem Dataset, Second Release (2024)

The aggregated data may be found here: https://williamslaro.github.io/talks/dataset2. Previously, the individual data was used, necessitating much preprocessing.

In [34]:
issue_data = pd.read_csv("C:\\Users\\alexa\\Code\\GW DATS\\6103 12 Intro to Data Mining\\Final Project\\data\\MIPD-2024-Aggregate\\MIPDV2-All-year.csv",encoding="ANSI",
                         index_col=["year","problem"])
issue_data.head(20)

cat  numsurveys  numrespondents       perc         catname  \
year problem                                                               
1939 1          1           3            8989  45.401001  MIPD (General)   
     2          1           3            8989   6.861000  MIPD (General)   
     3          1           3            8989   0.200000  MIPD (General)   
     4          1           3            8989   0.554000  MIPD (General)   
     5          1           3            8989   4.629000  MIPD (General)   
     6          1           3            8989  33.779999  MIPD (General)   
     7          1           3            8989   0.056000  MIPD (General)   
     8          1           3            8989   0.044000  MIPD (General)   
     9          1           3            8989   1.432000  MIPD (General)   
     10         1           3            8989   4.123000  MIPD (General)   
     11         1           3            8989   0.700000  MIPD (General)   
     12         1           3            8989   0.168000  MIPD (General)   
     13         1           3            8989   1.461000  MIPD (General)   
     14         1           3            8989   0.078000  MIPD (General)   
     15         1           3            8989   0.512000  MIPD (General)   
1940 1          1           1            2907  24.200001  MIPD (General)   
     2          1           1            2907   3.474000  MIPD (General)   
     3          1           1            2907   0.069000  MIPD (General)   
     4          1           1            2907   0.172000  MIPD (General)   
     5          1           1            2907  13.152000  MIPD (General)   

                                             catdesc  \
year problem                                           
1939 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   
     6        general problem categories in the MIPD   
     7        general problem categories in the MIPD   
     8        general problem categories in the MIPD   
     9        general problem categories in the MIPD   
     10       general problem categories in the MIPD   
     11       general problem categories in the MIPD   
     12       general problem categories in the MIPD   
     13       general problem categories in the MIPD   
     14       general problem categories in the MIPD   
     15       general problem categories in the MIPD   
1940 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   

                                   problemname  \
year problem                                     
1939 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   
     6                          Foreign Policy   
     7        International Economic Relations   
     8                             Environment   
     9                                Morality   
     10                               Politics   
     11                           Youth Issues   
     12                                 Groups   
     13                          Other and All   
     14                                    NaN   
     15                     Don't Know/Refused   
1940 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   

       

Limit to relevant years

In [118]:
issue_data = issue_data.loc[[2017,2018]]
issue_data

cat  numsurveys  numrespondents       perc         catname  \
year problem                                                               
2017 1          1           2            1999  19.669001  MIPD (General)   
     2          1           2            1999  16.531000  MIPD (General)   
     3          1           2            1999   4.969000  MIPD (General)   
     4          1           2            1999   1.275000  MIPD (General)   
     5          1           2            1999  15.047000  MIPD (General)   
     6          1           2            1999  11.080000  MIPD (General)   
     7          1           2            1999   0.222000  MIPD (General)   
     8          1           2            1999   2.157000  MIPD (General)   
     9          1           2            1999   5.814000  MIPD (General)   
     10         1           2            1999  14.678000  MIPD (General)   
     11         1           2            1999   0.426000  MIPD (General)   
     12         1           2            1999   0.000000  MIPD (General)   
     13         1           2            1999   3.186000  MIPD (General)   
     14         1           2            1999   0.269000  MIPD (General)   
     15         1           2            1999   4.676000  MIPD (General)   
2018 1          1           2            1983  14.319000  MIPD (General)   
     2          1           2            1983  11.224000  MIPD (General)   
     3          1           2            1983   8.407000  MIPD (General)   
     4          1           2            1983   3.457000  MIPD (General)   
     5          1           2            1983  15.009000  MIPD (General)   
     6          1           2            1983  12.685000  MIPD (General)   
     7          1           2            1983   0.282000  MIPD (General)   
     8          1           2            1983   1.959000  MIPD (General)   
     9          1           2            1983   5.000000  MIPD (General)   
     10         1           2            1983  17.754000  MIPD (General)   
     11         1           2            1983   0.707000  MIPD (General)   
     12         1           2            1983   0.000000  MIPD (General)   
     13         1           2            1983   4.345000  MIPD (General)   
     14         1           2            1983   0.125000  MIPD (General)   
     15         1           2            1983   4.728000  MIPD (General)   

                                             catdesc  \
year problem                                           
2017 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   
     6        general problem categories in the MIPD   
     7        general problem categories in the MIPD   
     8        general problem categories in the MIPD   
     9        general problem categories in the MIPD   
     10       general problem categories in the MIPD   
     11       general problem categories in the MIPD   
     12       general problem categories in the MIPD   
     13       general problem categories in the MIPD   
     14       general problem categories in the MIPD   
     15       general problem categories in the MIPD   
2018 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   
     6        general problem categories in the MIPD   
     7        general problem categories in the MIPD   
     8        general problem categories in the MIPD   
     9        general problem categories in the MIPD   
     10       general problem categories in the MIPD   
     11       general problem categories in the MIPD   
     12      

### Merge the Datasets

In [119]:
msa_fips_crosswalk = pd.read_excel("C:\\Users\\alexa\\Code\\GW DATS\\6103 12 Intro to Data Mining\\Final Project\\data\\list1_2023.xlsx",skiprows=2,dtype=str)
msa_fips_crosswalk.head()

,CBSA Code,Metropolitan Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,Central/Outlying County
0,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Brown County,South Dakota,46,013,Central
1,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Edmunds County,South Dakota,46,045,Outlying
2,10140,NaN,NaN,"Aberdeen, WA",Micropolitan Statistical Area,NaN,NaN,Grays Harbor County,Washington,53,027,Central
3,10180,NaN,101,"Abilene, TX",Metropolitan Statistical Area,NaN,"Abilene-Sweetwater, TX",Callahan County,Texas,48,059,Outlying
4,10180,NaN,101,"Abilene, TX",Metropolitan Statistical Area,NaN,"Abilene-Sweetwater, TX",Jones County,Texas,48,253,Outlying


In [ ]:
msa_fips_crosswalk["FIPS Code"] = msa_fips_crosswalk["FIPS State Code"] + msa_fips_crosswalk["FIPS County Code"]

,CBSA Code,Metropolitan Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,Central/Outlying County,FIPS Code
1915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1916,Note: The Office of Management and Budget's (O...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1917,"Source: File prepared by U.S. Census Bureau, P...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Drop NaN FIPS codes in preparation for merging.

In [163]:
msa_fips_crosswalk = msa_fips_crosswalk[msa_fips_crosswalk["FIPS Code"].notna()]
msa_fips_crosswalk["FIPS Code"] = msa_fips_crosswalk["FIPS Code"].astype(int)
msa_fips_crosswalk["FIPS Code"]

0       46013
1       46045
2       53027
3       48059
4       48253
        ...  
1910     6101
1911     6115
1912     4027
1913    39119
1914    48505
Name: FIPS Code, Length: 1915, dtype: int64

In [166]:
crowd_data_msa = crowd_data.merge(msa_fips_crosswalk[["FIPS Code", "CBSA Code"]], how="left", left_on="fips_code", right_on="FIPS Code")
crowd_data_msa.head()


,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,chemical_agents,notes,lat,lon,resolved_locality,resolved_county,resolved_state,fips_code,FIPS Code,CBSA Code
0,2017-01-01,Washington,DC,Lafayette Square Park,0.0,vigil,NaN,NaN,"for banning nuclear weapons, for peace",0.0,...,0.0,White House Peace Vigil continuous since June ...,38.907192,-77.036871,Washington,District of Columbia,DC,11001,11001.0,47900
1,2017-01-01,Mankato,MN,NaN,0.0,vigil,NaN,Peace Vigil Mankato,for peace,0.0,...,0.0,every Sunday since 2001,44.163578,-93.999400,Mankato,Blue Earth County,MN,27013,27013.0,31860
2,2017-01-01,Minneapolis,MN,U.S. Bank Stadium,0.0,protest; banner drop,NaN,general protestors,"against the Dakota Access Pipeline, for indige...",1.0,...,NaN,hung banner from stadium roof during NFL game,44.977753,-93.265011,Minneapolis,Hennepin County,MN,27053,27053.0,33460
3,2017-01-01,Little Compton,RI,Town Green,0.0,vigil,NaN,Sakonnet Peace Alliance,"for peace, for gun control, for climate action",1.0,...,0.0,every Sunday since 2003,41.510103,-71.171156,Little Compton,Newport County,RI,44005,44005.0,39300
4,2017-01-01,Oak Ridge,TN,Y-12 National Security Complex,0.0,vigil,NaN,Oak Ridge Environmental Peace Alliance,for abolishing nuclear weapons,0.0,...,0.0,every Sunday since the late 1990s,36.010356,-84.269645,Oak Ridge,Anderson County,TN,47001,47001.0,28940
